<a href="https://colab.research.google.com/github/amr552/Deep-Learning/blob/NLP/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Genetic Algorithm

In [ ]:
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load your image dataset (replace with your actual data loading code)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = '/content/drive/MyDrive/datasets/plant_disease/Train/Train'
val_dataset = '/content/drive/MyDrive/datasets/plant_disease/Validation/Validation'
test_dataset = '/content/drive/MyDrive/datasets/plant_disease/Test/Test'

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(28, 28),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    val_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

def create_cnn_model(params):
    learning_rate, num_filters, kernel_size, stride, pooling_type, pooling_size, batch_size, optimizer_name = params

    if pooling_type == 'max':
        PoolingLayer = MaxPooling2D
    else:
        PoolingLayer = AveragePooling2D

    optimizers = {
        'adam': tf.keras.optimizers.Adam(learning_rate),
        'sgd': tf.keras.optimizers.SGD(learning_rate),
        'rmsprop': tf.keras.optimizers.RMSprop(learning_rate)
    }

    optimizer = optimizers[optimizer_name]

    model = Sequential()
    model.add(Conv2D(num_filters, (kernel_size, kernel_size), strides=stride, activation='relu', input_shape=(28, 28, 3)))
    model.add(Conv2D(num_filters, (kernel_size, kernel_size), strides=stride, activation='relu',padding='same'))
    model.add(PoolingLayer(pool_size=(pooling_size, pooling_size)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

def fitness(params):
    learning_rate, num_filters, kernel_size, stride, pooling_type, pooling_size, batch_size, optimizer_name = params
    model = create_cnn_model(params)

    # Fit model for a few epochs to evaluate
    model.fit(train_generator, epochs=1, batch_size=batch_size, verbose=0)  # Adjust training epochs as needed

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_generator, verbose=0)
    return accuracy

def genetic_algorithm():
    # Define hyperparameter ranges (adjust these as needed)
    learning_rate_range = (0.001, 0.1)
    num_filters_range = (16, 64)
    kernel_size_range = (3, 5)
    stride_range = (1, 2)
    pooling_type_options = ['max', 'avg']
    pooling_size_range = (2, 3)
    batch_size_range = (16, 64)
    optimizer_names = ['adam', 'sgd', 'rmsprop']

    # Generate initial population of solutions
    solutions = []
    for _ in range(100):
        learning_rate = random.uniform(*learning_rate_range)
        num_filters = random.randint(*num_filters_range)
        kernel_size = random.randint(*kernel_size_range)
        stride = random.randint(*stride_range)
        pooling_type = random.choice(pooling_type_options)
        pooling_size = random.randint(*pooling_size_range)
        batch_size = random.randint(*batch_size_range)
        optimizer_name = random.choice(optimizer_names)
        solutions.append((learning_rate, num_filters, kernel_size, stride, pooling_type, pooling_size, batch_size, optimizer_name))

    for generation in range(100):
        ranked_solutions = []
        for solution in solutions:
            fitness_value = fitness(solution)
            ranked_solutions.append((fitness_value, solution))

        ranked_solutions.sort(key=lambda x: x[0], reverse=True)


        print(f'=== Gen {generation} best solutions ===')
        print(ranked_solutions[0])

        if ranked_solutions[0][0] > 0.99:
            break

        # Selection (choose best solutions for breeding)
        best_solutions = ranked_solutions[:20]

        # Crossover (combine genes from parents to create offspring)
        new_generation = []
        for _ in range(100):
            parent1 = random.choice(best_solutions)[1]
            parent2 = random.choice(best_solutions)[1]
            offspring = (
                random.choice([parent1[0], parent2[0]]),  # Learning rate
                random.choice([parent1[1], parent2[1]]),  # Number of filters
                random.choice([parent1[2], parent2[2]]),  # Kernel size
                random.choice([parent1[3], parent2[3]]),  # Stride
                random.choice([parent1[4], parent2[4]]),  # Pooling type
                random.choice([parent1[5], parent2[5]]),  # Pooling size
                random.choice([parent1[6], parent2[6]]),  # Batch size
                random.choice([parent1[7], parent2[7]])   # Optimizer name
            )
            new_generation.append(offspring)

        # Mutation (randomly alter some offspring)
        for i in range(len(new_generation)):
            if random.random() < 0.1:  # 10% mutation rate
                new_generation[i] = (
                    random.uniform(*learning_rate_range),
                    random.randint(*num_filters_range),
                    random.randint(*kernel_size_range),
                    random.randint(*stride_range),
                    random.choice(pooling_type_options),
                    random.randint(*pooling_size_range),
                    random.randint(*batch_size_range),
                    random.choice(optimizer_names)
                )

        solutions = new_generation

    # Return the best solution
    best_solution = ranked_solutions[0]
    print(f'Best solution: {best_solution[1]} with accuracy: {best_solution[0]}')

# Run the genetic algorithm
genetic_algorithm()


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


=== Gen 0 best solutions ===
(0.6000000238418579, (0.05162399086507868, 64, 5, 1, 'max', 3, 38, 'sgd'))


In [ ]:
import random
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load your image dataset (replace with your actual data loading code)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset='/content/drive/MyDrive/datasets/plant_disease/Train/Train'
val_dataset='/content/drive/MyDrive/datasets/plant_disease/Validation/Validation'
test_dataset='/content/drive/MyDrive/datasets/plant_disease/Test/Test'

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(28, 28),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    val_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

def create_cnn_model(learning_rate, num_filters, kernel_size):
    model = Sequential()
    model.add(Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate), metrics=['accuracy'])
    return model

def fitness(params):
    learning_rate, num_filters, kernel_size = params
    model = create_cnn_model(learning_rate, num_filters, kernel_size)

    # Fit model for one epoch to evaluate
    model.fit(train_generator, epochs=5, verbose=0)  # Adjust training epochs as needed

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_generator, verbose=0)
    return accuracy


def genetic_algorithm():
    # Define hyperparameter ranges (adjust these as needed)
    learning_rate_range = (0.001, 0.1)
    num_filters_range = (16, 64)
    kernel_size_range = (3, 5)

    # Generate initial population of solutions
    solutions = []
    for _ in range(100):
        learning_rate = random.uniform(*learning_rate_range)
        num_filters = random.randint(*num_filters_range)
        kernel_size = random.randint(*kernel_size_range)
        solutions.append((learning_rate, num_filters, kernel_size))

    for generation in range(100):
        ranked_solutions = []
        for solution in solutions:
            fitness_value = fitness(solution)
            ranked_solutions.append((fitness_value, solution))

        ranked_solutions.sort(key=lambda x: x[0], reverse=True)

        print(f'===Gen {generation} best solutions ===')
        print(ranked_solutions[0])

        if ranked_solutions[0][0] > 0.99:
            break

        # Selection (choose best solutions for breeding)
        best_solutions = ranked_solutions[:20]

        # Crossover (combine genes from parents to create offspring)
        new_generation = []
        for _ in range(100):
            parent1 = random.choice(best_solutions)[1]
            parent2 = random.choice(best_solutions)[1]
            offspring = (
                random.choice([parent1[0], parent2[0]]),  # Learning rate
                random.choice([parent1[1], parent2[1]]),  # Number of filters
                random.choice([parent1[2], parent2[2]])   # Kernel size
            )
            new_generation.append(offspring)

        # Mutation (randomly alter some offspring)
        for i in range(len(new_generation)):
            if random.random() < 0.1:  # 10% mutation rate
                new_generation[i] = (
                    random.uniform(*learning_rate_range),
                    random.randint(*num_filters_range),
                    random.randint(*kernel_size_range)
                )

        solutions = new_generation

    # Return the best solution
    best_solution = ranked_solutions[0]
    print(f'Best solution: {best_solution[1]} with accuracy: {best_solution[0]}')

# Run the genetic algorithm
genetic_algorithm()



Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


===Gen 0 best solutions ===
(0.6666666865348816, (0.004003320733649794, 31, 3))


#PSO Algorithm for getting the ideal parameters of a simple CNN model

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

# Load your image dataset (replace with your actual data loading code)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = '/content/drive/MyDrive/datasets/plant_disease/Train/Train'
val_dataset = '/content/drive/MyDrive/datasets/plant_disease/Validation/Validation'
test_dataset = '/content/drive/MyDrive/datasets/plant_disease/Test/Test'

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(28, 28),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    val_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

def create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name):
    model = Sequential()
    kernel_size = int(kernel_size)
    num_filters = int(num_filters)

    model.add(Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 3)))

    if pooling_type == 'max':
        model.add(MaxPooling2D((2, 2)))
    elif pooling_type == 'avg':
        model.add(AveragePooling2D((2, 2)))
    else:
        raise ValueError("Invalid pooling type")

    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D((3, 3)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def fitness(params):
    learning_rate, num_filters, kernel_size, pooling_type, optimizer_name = params
    model = create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name)

    # Fit the model for one epoch (adjust as needed)
    model.fit(train_generator, epochs=1, verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_generator, verbose=0)
    return accuracy  # Only return the accuracy metric

def particle_swarm_optimization():
    # Define hyperparameter ranges (adjust these as needed)
    learning_rate_range = (0.001, 0.1)
    num_filters_range = (16, 64)
    kernel_size_range = (3, 5)
    pooling_type_range = ('max', 'avg')
    optimizer_range = ('adam', 'rmsprop', 'sgd')

    # Initialize particles
    num_particles = 10  # Reduced number for testing
    particles = []
    velocities = []
    for _ in range(num_particles):
        particle = [
            random.uniform(*learning_rate_range),
            random.randint(*num_filters_range),
            random.randint(*kernel_size_range),
            random.choice(pooling_type_range),
            random.choice(optimizer_range)
        ]
        velocity = [
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1)
        ]
        particles.append(particle)
        velocities.append(velocity)

    # Personal best and global best
    pbest = particles.copy()
    pbest_fitness = [fitness(p) for p in particles]
    gbest = particles[np.argmax(pbest_fitness)]
    gbest_fitness = max(pbest_fitness)

    # Particle Swarm Optimization
    for iteration in range(10):  # Reduced iterations for testing
        for i in range(num_particles):
            # Update velocity
            w = 0.9
            c1 = 2
            c2 = 2
            r1 = random.random()
            r2 = random.random()
            velocities[i] = [
                w * velocities[i][0] + c1 * r1 * (pbest[i][0] - particles[i][0]) + c2 * r2 * (gbest[0] - particles[i][0]),
                w * velocities[i][1] + c1 * r1 * (pbest[i][1] - particles[i][1]) + c2 * r2 * (gbest[1] - particles[i][1]),
                w * velocities[i][2] + c1 * r1 * (pbest[i][2] - particles[i][2]) + c2 * r2 * (gbest[2] - particles[i][2]),
                velocities[i][3],  # No change for categorical params
                velocities[i][4]   # No change for categorical params
            ]

            # Update particle position
            particles[i] = [
                particles[i][0] + velocities[i][0],
                int(particles[i][1] + velocities[i][1]),  # Ensure integer values
                int(particles[i][2] + velocities[i][2]),  # Ensure integer values
                particles[i][3],
                particles[i][4]
            ]

            # Ensure the hyperparameters are within valid ranges
            particles[i][0] = max(min(particles[i][0], learning_rate_range[1]), learning_rate_range[0])
            particles[i][1] = max(min(particles[i][1], num_filters_range[1]), num_filters_range[0])
            particles[i][2] = max(min(particles[i][2], kernel_size_range[1]), kernel_size_range[0])

            # Evaluate particle fitness
            particle_fitness = fitness(particles[i])

            # Update personal best
            if particle_fitness > pbest_fitness[i]:
                pbest[i] = particles[i]
                pbest_fitness[i] = particle_fitness

            # Update global best
            if particle_fitness > gbest_fitness:
                gbest = particles[i]
                gbest_fitness = particle_fitness

        print(f'Iteration {iteration}, Best accuracy: {gbest_fitness:.4f}')

        if gbest_fitness > 0.99:  # Arbitrary stopping condition for testing
            break

    print(f'Best solution: {gbest} with accuracy: {gbest_fitness:.4f}')

# Run the Particle Swarm Optimization
particle_swarm_optimization()


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Iteration 0, Best accuracy: 0.5333
Iteration 1, Best accuracy: 0.6500
Iteration 2, Best accuracy: 0.6500
Iteration 3, Best accuracy: 0.6500
Iteration 4, Best accuracy: 0.6500
Iteration 5, Best accuracy: 0.6500
Iteration 6, Best accuracy: 0.6500
Iteration 7, Best accuracy: 0.6500
Iteration 8, Best accuracy: 0.6500
Iteration 9, Best accuracy: 0.6667
Best solution: [0.03888646513457217, 64, 3, 'max', 'rmsprop'] with accuracy: 0.6667


In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

# Load your image dataset (replace with your actual data loading code)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = '/content/drive/MyDrive/datasets/plant_disease/Train/Train'
val_dataset = '/content/drive/MyDrive/datasets/plant_disease/Validation/Validation'
test_dataset = '/content/drive/MyDrive/datasets/plant_disease/Test/Test'

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(28, 28),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    val_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

# Define the CNN model creation function
def create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name):
    model = Sequential()
    kernel_size = int(kernel_size)
    num_filters = int(num_filters)

    model.add(Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 3)))

    if pooling_type == 'max':
        model.add(MaxPooling2D((2, 2)))
    elif pooling_type == 'avg':
        model.add(AveragePooling2D((2, 2)))
    else:
        raise ValueError("Invalid pooling type")

    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D((3, 3)))
    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the fitness function
def fitness(params):
    learning_rate, num_filters, kernel_size, pooling_type, optimizer_name = params
    model = create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name)

    # Fit the model for one epoch (adjust as needed)
    model.fit(train_generator, epochs=1, verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_generator, verbose=0)
    return accuracy  # Only return the accuracy metric

# Define the Particle Swarm Optimization (PSO) function
def particle_swarm_optimization():
    # Define hyperparameter ranges (adjust these as needed)
    learning_rate_range = (0.001, 0.1)
    num_filters_range = (16, 64)
    kernel_size_range = (3, 5)
    pooling_type_range = ('max', 'avg')
    optimizer_range = ('adam', 'rmsprop', 'sgd')

    # Initialize particles
    num_particles = 10  # Reduced number for testing
    particles = []
    velocities = []
    for _ in range(num_particles):
        particle = [
            random.uniform(*learning_rate_range),
            random.randint(*num_filters_range),
            random.randint(*kernel_size_range),
            random.choice(pooling_type_range),
            random.choice(optimizer_range)
        ]
        velocity = [
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1)
        ]
        particles.append(particle)
        velocities.append(velocity)

    # Personal best and global best
    pbest = particles.copy()
    pbest_fitness = [fitness(p) for p in particles]
    gbest = particles[np.argmax(pbest_fitness)]
    gbest_fitness = max(pbest_fitness)

    # Particle Swarm Optimization
    for iteration in range(10):  # Reduced iterations for testing
        for i in range(num_particles):
            # Update velocity
            w = 0.9
            c1 = 2
            c2 = 2
            r1 = random.random()
            r2 = random.random()
            velocities[i] = [
                w * velocities[i][0] + c1 * r1 * (pbest[i][0] - particles[i][0]) + c2 * r2 * (gbest[0] - particles[i][0]),
                w * velocities[i][1] + c1 * r1 * (pbest[i][1] - particles[i][1]) + c2 * r2 * (gbest[1] - particles[i][1]),
                w * velocities[i][2] + c1 * r1 * (pbest[i][2] - particles[i][2]) + c2 * r2 * (gbest[2] - particles[i][2]),
                velocities[i][3],  # No change for categorical params
                velocities[i][4]   # No change for categorical params
            ]

            # Update particle position
            particles[i] = [
                particles[i][0] + velocities[i][0],
                int(particles[i][1] + velocities[i][1]),  # Ensure integer values
                int(particles[i][2] + velocities[i][2]),  # Ensure integer values
                particles[i][3],
                particles[i][4]
            ]

            # Ensure the hyperparameters are within valid ranges
            particles[i][0] = max(min(particles[i][0], learning_rate_range[1]), learning_rate_range[0])
            particles[i][1] = max(min(particles[i][1], num_filters_range[1]), num_filters_range[0])
            particles[i][2] = max(min(particles[i][2], kernel_size_range[1]), kernel_size_range[0])

            # Evaluate particle fitness
            particle_fitness = fitness(particles[i])

            # Update personal best
            if particle_fitness > pbest_fitness[i]:
                pbest[i] = particles[i]
                pbest_fitness[i] = particle_fitness

            # Update global best
            if particle_fitness > gbest_fitness:
                gbest = particles[i]
                gbest_fitness = particle_fitness

        print(f'Iteration {iteration}, Best accuracy: {gbest_fitness:.4f}')
        print(f'Accuracy {gbest_fitness:.4f} of parameters {particles}\n')

        if gbest_fitness > 0.99:  # Arbitrary stopping condition for testing
            break

    print(f'Best solution: {gbest} with accuracy: {gbest_fitness:.4f}')

# Run the Particle Swarm Optimization
particle_swarm_optimization()


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


Iteration 0, Best accuracy: 0.5833
Accuracy 0.5833 of parameters [[0.1, 29, 5, 'avg', 'sgd'], [0.1, 28, 3, 'avg', 'adam'], [0.001, 29, 3, 'avg', 'adam'], [0.001, 30, 4, 'max', 'rmsprop'], [0.1, 61, 5, 'max', 'rmsprop'], [0.1, 19, 3, 'avg', 'rmsprop'], [0.1, 28, 3, 'max', 'rmsprop'], [0.1, 32, 4, 'max', 'adam'], [0.1, 21, 3, 'avg', 'rmsprop'], [0.001, 32, 3, 'max', 'rmsprop']]
Iteration 1, Best accuracy: 0.5833
Accuracy 0.5833 of parameters [[0.1, 31, 5, 'avg', 'sgd'], [0.09356995056332348, 28, 3, 'avg', 'adam'], [0.001, 16, 3, 'avg', 'adam'], [0.001, 22, 3, 'max', 'rmsprop'], [0.1, 16, 3, 'max', 'rmsprop'], [0.1, 64, 5, 'avg', 'rmsprop'], [0.08965386785905946, 24, 4, 'max', 'rmsprop'], [0.09148799288351903, 42, 4, 'max', 'adam'], [0.1, 46, 5, 'avg', 'rmsprop'], [0.001, 16, 3, 'max', 'rmsprop']]
Iteration 2, Best accuracy: 0.5833
Accuracy 0.5833 of parameters [[0.1, 27, 4, 'avg', 'sgd'], [0.03994330304062848, 29, 4, 'avg', 'adam'], [0.001, 16, 3, 'avg', 'adam'], [0.001, 34, 4, 'max', 'r

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

# Load your image dataset (replace with your actual data loading code)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = '/content/drive/MyDrive/datasets/plant_disease/Train/Train'
val_dataset = '/content/drive/MyDrive/datasets/plant_disease/Validation/Validation'
test_dataset = '/content/drive/MyDrive/datasets/plant_disease/Test/Test'

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(28, 28),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    val_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

def create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name):
    model = Sequential()
    kernel_size = int(kernel_size)
    num_filters = int(num_filters)

    model.add(Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 3)))

    if pooling_type == 'max':
        model.add(MaxPooling2D((2, 2)))
    elif pooling_type == 'avg':
        model.add(AveragePooling2D((2, 2)))
    else:
        raise ValueError("Invalid pooling type")

    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D((3, 3)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def fitness(params):
    learning_rate, num_filters, kernel_size, pooling_type, optimizer_name = params
    model = create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name)

    # Fit the model for one epoch (adjust as needed)
    model.fit(train_generator, epochs=1, verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_generator, verbose=0)
    return accuracy  # Only return the accuracy metric

def particle_swarm_optimization():
    # Define hyperparameter ranges (adjust these as needed)
    learning_rate_range = (0.001, 0.1)
    num_filters_range = (16, 64)
    kernel_size_range = (3, 5)
    pooling_type_range = ('max', 'avg')
    optimizer_range = ('adam', 'rmsprop', 'sgd')

    # Initialize particles
    num_particles = 10  # Reduced number for testing
    particles = []
    velocities = []
    for _ in range(num_particles):
        particle = [
            random.uniform(*learning_rate_range),
            random.randint(*num_filters_range),
            random.randint(*kernel_size_range),
            random.choice(pooling_type_range),
            random.choice(optimizer_range)
        ]
        velocity = [
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1)
        ]
        particles.append(particle)
        velocities.append(velocity)

    # Personal best and global best
    pbest = particles.copy()
    pbest_fitness = [fitness(p) for p in particles]
    gbest = particles[np.argmax(pbest_fitness)]
    gbest_fitness = max(pbest_fitness)

    # Particle Swarm Optimization
    for iteration in range(10):  # Reduced iterations for testing
        for i in range(num_particles):
            # Update velocity
            w = 0.9
            c1 = 2
            c2 = 2
            r1 = random.random()
            r2 = random.random()
            velocities[i] = [
                w * velocities[i][0] + c1 * r1 * (pbest[i][0] - particles[i][0]) + c2 * r2 * (gbest[0] - particles[i][0]),
                w * velocities[i][1] + c1 * r1 * (pbest[i][1] - particles[i][1]) + c2 * r2 * (gbest[1] - particles[i][1]),
                w * velocities[i][2] + c1 * r1 * (pbest[i][2] - particles[i][2]) + c2 * r2 * (gbest[2] - particles[i][2]),
                velocities[i][3],  # No change for categorical params
                velocities[i][4]   # No change for categorical params
            ]

            # Update particle position
            particles[i] = [
                particles[i][0] + velocities[i][0],
                int(particles[i][1] + velocities[i][1]),  # Ensure integer values
                int(particles[i][2] + velocities[i][2]),  # Ensure integer values
                particles[i][3],
                particles[i][4]
            ]

            # Ensure the hyperparameters are within valid ranges
            particles[i][0] = max(min(particles[i][0], learning_rate_range[1]), learning_rate_range[0])
            particles[i][1] = max(min(particles[i][1], num_filters_range[1]), num_filters_range[0])
            particles[i][2] = max(min(particles[i][2], kernel_size_range[1]), kernel_size_range[0])

            # Evaluate particle fitness
            particle_fitness = fitness(particles[i])

            # Update personal best
            if particle_fitness > pbest_fitness[i]:
                pbest[i] = particles[i]
                pbest_fitness[i] = particle_fitness

            # Update global best
            if particle_fitness > gbest_fitness:
                gbest = particles[i]
                gbest_fitness = particle_fitness

        print(f'Iteration {iteration}, Best accuracy: {gbest_fitness:.4f}')

        if gbest_fitness > 0.99:  # Arbitrary stopping condition for testing
            break

    print(f'Best solution: {gbest} with accuracy: {gbest_fitness:.4f}')

# Run the Particle Swarm Optimization
particle_swarm_optimization()


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Iteration 0, Best accuracy: 0.6333
Iteration 1, Best accuracy: 0.6333
Iteration 2, Best accuracy: 0.7000
Iteration 3, Best accuracy: 0.7000
Iteration 4, Best accuracy: 0.7000
Iteration 5, Best accuracy: 0.7000
Iteration 6, Best accuracy: 0.7000
Iteration 7, Best accuracy: 0.7000
Iteration 8, Best accuracy: 0.7000
Iteration 9, Best accuracy: 0.7000
Best solution: [0.001, 16, 3, 'max', 'adam'] with accuracy: 0.7000


In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

# Load your image dataset (replace with your actual data loading code)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = '/content/drive/MyDrive/datasets/plant_disease/Train/Train'
val_dataset = '/content/drive/MyDrive/datasets/plant_disease/Validation/Validation'
test_dataset = '/content/drive/MyDrive/datasets/plant_disease/Test/Test'

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(28, 28),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    val_dataset,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical'
)

def create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name):
    model = Sequential()
    kernel_size = int(kernel_size)
    num_filters = int(num_filters)

    model.add(Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 3)))

    if pooling_type == 'max':
        model.add(MaxPooling2D((2, 2)))
    elif pooling_type == 'avg':
        model.add(AveragePooling2D((2, 2)))
    else:
        raise ValueError("Invalid pooling type")

    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D((3, 3)))
    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def fitness(params):
    learning_rate, num_filters, kernel_size, pooling_type, optimizer_name = params
    model = create_cnn_model(learning_rate, num_filters, kernel_size, pooling_type, optimizer_name)

    # Fit the model for one epoch (adjust as needed)
    model.fit(train_generator, epochs=1, verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_generator, verbose=0)
    return accuracy  # Only return the accuracy metric

def particle_swarm_optimization():
    # Define hyperparameter ranges (adjust these as needed)
    learning_rate_range = (0.001, 0.1)
    num_filters_range = (16, 64)
    kernel_size_range = (3, 5)
    pooling_type_range = ('max', 'avg')
    optimizer_range = ('adam', 'rmsprop', 'sgd')

    # Initialize particles
    num_particles = 10  # Reduced number for testing
    particles = []
    velocities = []
    for _ in range(num_particles):
        particle = [
            random.uniform(*learning_rate_range),
            random.randint(*num_filters_range),
            random.randint(*kernel_size_range),
            random.choice(pooling_type_range),
            random.choice(optimizer_range)
        ]
        velocity = [
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1),
            random.uniform(-1, 1)
        ]
        particles.append(particle)
        velocities.append(velocity)

    # Personal best and global best
    pbest = particles.copy()
    pbest_fitness = [fitness(p) for p in particles]
    gbest = particles[np.argmax(pbest_fitness)]
    gbest_fitness = max(pbest_fitness)

    # Particle Swarm Optimization
    for iteration in range(10):  # Reduced iterations for testing
        for i in range(num_particles):
            # Update velocity
            w = 0.9
            c1 = 2
            c2 = 2
            r1 = random.random()
            r2 = random.random()
            velocities[i] = [
                w * velocities[i][0] + c1 * r1 * (pbest[i][0] - particles[i][0]) + c2 * r2 * (gbest[0] - particles[i][0]),
                w * velocities[i][1] + c1 * r1 * (pbest[i][1] - particles[i][1]) + c2 * r2 * (gbest[1] - particles[i][1]),
                w * velocities[i][2] + c1 * r1 * (pbest[i][2] - particles[i][2]) + c2 * r2 * (gbest[2] - particles[i][2]),
                velocities[i][3],
                velocities[i][4]
            ]
              '''
              velocities[i] is the velocity of particle i
              w is the inertia weight
              c1 and c2 are the cognitive and social acceleration coefficients
              r1 and r2 are random numbers between 0 and 1
              pbest[i] is the personal best of particle i
              gbest is the global best
              particles[i] is the position of particle i

              '''
            # Update particle position
            particles[i] = [
                particles[i][0] + velocities[i][0],
                int(particles[i][1] + velocities[i][1]),
                int(particles[i][2] + velocities[i][2]),
                particles[i][3],
                particles[i][4]
            ]

            # Ensure the hyperparameters are within valid ranges
            particles[i][0] = max(min(particles[i][0], learning_rate_range[1]), learning_rate_range[0])
            particles[i][1] = max(min(particles[i][1], num_filters_range[1]), num_filters_range[0])
            particles[i][2] = max(min(particles[i][2], kernel_size_range[1]), kernel_size_range[0])

            # Evaluate particle fitness
            particle_fitness = fitness(particles[i])

            # Update personal best
            if particle_fitness > pbest_fitness[i]:
                pbest[i] = particles[i]
                pbest_fitness[i] = particle_fitness

            # Update global best
            if particle_fitness > gbest_fitness:
                gbest = particles[i]
                gbest_fitness = particle_fitness

        print(f'Iteration {iteration}, Best accuracy: {gbest_fitness:.4f}')

        if gbest_fitness > 0.99:  # Arbitrary stopping condition for testing
            break

    print(f'Best solution: {gbest} with accuracy: {gbest_fitness:.4f}')

# Run the Particle Swarm Optimization
particle_swarm_optimization()


Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


Iteration 0, Best accuracy: 0.5667
Iteration 1, Best accuracy: 0.5667
Iteration 2, Best accuracy: 0.5667
Iteration 3, Best accuracy: 0.5667
Iteration 4, Best accuracy: 0.6000
Iteration 5, Best accuracy: 0.6000
Iteration 6, Best accuracy: 0.6000
Iteration 7, Best accuracy: 0.6000
Iteration 8, Best accuracy: 0.6000
Iteration 9, Best accuracy: 0.6000
Best solution: [0.001, 16, 3, 'avg', 'rmsprop'] with accuracy: 0.6000
